# PostgreSQL Workflow

In [1]:
import psycopg2 as pg
import pandas as pd

My idea for this workflow is that we can separate out feature engineering and modeling into more modular pieces. First, we can all connect to the same postgres database for a single source truth. Note if you want to just explore the data in a GUI environment you'll want to download pgadmin4 and use these same credentials

In [6]:
# establish connection to postgres
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

Here's an example of just quickly pulling the raw data that was loaded. There are several other ways to pull data also that doesn't require directly using Pandas

In [3]:
query1 = '''SELECT * FROM "Teams"'''
example1 = pd.read_sql_query(query1, conn)

In [4]:
example1.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [5]:
conn.close()

Let's take a look at our features table. (keep in mind this is just an example table with a subset of the data for now)

In [ ]:
query2 = '''SELECT * FROM features_example LIMIT 20'''
example2 = pd.read_sql_query(query2, conn)
example2.head()

Next, say for example one of us came up with a good idea for a feature. We could create the feature in Python and push it up to the features table in postgres. For this particular example, I'm going to generate features based on columns from the features_example table -- but this data would more realistically come from some other source -- either loaded or external.

In [13]:
query3 = '''
SELECT "Season","DayNum", "Team", "Score", "OpponentScore"
FROM features_example'''

example3 = pd.read_sql_query(query3, conn)
example3['new_feature'] = example3['Score'] - example3['OpponentScore']
example3.head()

,Season,DayNum,Team,Score,OpponentScore,new_feature
0,1985,136,1233,58,59,-1
1,1985,136,1292,57,76,-19
2,1985,136,1323,79,70,9
3,1985,136,1235,64,75,-11
4,1985,136,1299,83,96,-13


Now that I've created a new feature, I want to push this to postgres so any of us can use the new feature to model on.

In [ ]:
# establish connection to postgres
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

query1 = '''ALTER TABLE features_example ADD COLUMN IF NOT EXISTS "NewFeature" INT'''
c = conn.cursor()
c.execute(query1)
conn.commit()
conn.close()

In [30]:
features_tuple = []
for i in example3.itertuples(index=False):
    features_tuple.append((int(i.new_feature), int(i.Season), int(i.DayNum), int(i.Team)))

In [31]:
from psycopg2.extras import execute_values

update = '''
        UPDATE features_example
          SET "NewFeature" = data.new_feature
          FROM (VALUES %s) AS data (
            new_feature,
            "Season",
            "DayNum",
            "Team"
          )
          WHERE features_example."Season" = data."Season"
            and features_example."DayNum" = data."DayNum"
            and features_example."Team" = data."Team"
          '''

In [32]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

c = conn.cursor()
execute_values(c, update, features_tuple)
conn.commit()
conn.close()